In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.touch_actions import TouchActions
from lxml import html
import os
import json
import numpy as np
import time
import tablib
import json

In [3]:
# Functions
def find_session_id(driver):
	# Find session_id
	url = driver.command_executor._url 
	session_id = driver.session_id  

	print("url: ", url)
	print("Session Id: ", session_id)
    
def organize_content(company, interested_field, field_content, description_info):
    Content = {"company": company, "content": "FOUND"}
    for interest in interested_field:
        try:
            idx = field_content.index(interest)
            Content[str(interest)]=field_content[idx+1]
        except:
            Content[str(interest)]='N/A'
    Content['description'] = description_info
    return Content

def found_company(driver, company):
    # FIND CONTENT CARD
    mat_card = driver.find_elements_by_xpath("/html[1]/body[1]/chrome[1]/div[1]/mat-sidenav-container[1]/mat-sidenav-content[1]/entity[1]/page-layout[1]/div[2]/div[1]/div[2]/div[1]/div[1]/entity-section[1]/section-layout[1]/mat-card[1]")
    content = mat_card[0].find_elements_by_class_name('section-layout-content')
    # FIND FIELD CONTENT
    field_content = content[0].text.split('\n')
    field_content = [cont.lower().strip() for cont in field_content]
    # FIND DESCRIPTION
    try: 
        description_info = content[0].find_elements_by_tag_name('description-card')[0].text
    except:
        description_info = "N/A"
    # ORGANIZE INFORMATION
    Content = organize_content(company, interested_field, field_content, description_info)
    return Content

In [4]:
driver = webdriver.Chrome('./chromedriver')
actions = ActionChains(driver)
find_session_id(driver)
# GO TO WEBSITE
driver.get('https://www.crunchbase.com/')
time.sleep(5)

url:  http://127.0.0.1:49613
Session Id:  67a14b047c2ceee5dbfa9d7f4c0b7185


In [5]:
with open('company_chris.txt') as f:
    company_list = f.read().split('\n')
interested_field = ['operating status', 'funding status', 'last funding type', 'company type', 'website' , 
                    'last funding type', 'categories']
Content_List = np.array([])
wait_time = 1.5
for idx_company, company_str in enumerate(company_list):
    # SEARCH COMPANY
    searchbar = driver.find_elements_by_xpath("//input[@id='mat-input-0']")
    searchbar[0].send_keys(company_str)
    searchbar[0].clear()
    searchbar[0].send_keys(Keys.RETURN)
    time.sleep(wait_time)
    
    search_list = driver.find_elements_by_xpath("/html[1]/body[1]/chrome[1]/div[1]/mat-sidenav-container[1]/mat-sidenav-content[1]/search[1]/page-layout[1]/div[1]/div[1]/form[1]/div[2]/results[1]/div[1]/div[1]/div[3]/sheet-grid[1]/div[1]/div[1]/grid-body[1]/div[1]/div[1]")
    found = False
    try:
        for company_ele in search_list[0].find_elements_by_tag_name("grid-row"):
            verify_company = company_ele.text.split('\n')[1]
            is_company = verify_company.lower().find(company_str.lower())
            print("CHECKING :", company_ele.text.split('\n')[1])
            print("IS_COMPANY: ", is_company)
            if is_company >= 0:
                print(company_str, " FOUND")
                found = True
                # CLICK FIRST LINK
                hyperlink_str = "//a[@title='" + verify_company + "']"
                driver.find_element_by_xpath(hyperlink_str).click()
                time.sleep(wait_time)
                Content = found_company(driver, verify_company)
                break
            else:
                print(company_str, " NOT FOUND")  
                continue
            print("")
        if not found:
            Content = {"company": company_str, "content": "NOT FOUND"}
        Content_List = np.append(Content_List, Content)
        print("______________________________________")
    except:
        print("COULDN'T FIND ", company_str)
        print("______________________________________")
    time.sleep(wait_time)

CHECKING : Ipsyt
IS_COMPANY:  0
Ipsyt  FOUND
______________________________________
CHECKING : Iris Technologies
IS_COMPANY:  0
Iris Technologies  FOUND
______________________________________
CHECKING : IRONCloud Industries
IS_COMPANY:  0
IRONCloud Industries  FOUND
______________________________________
CHECKING : IsoFit
IS_COMPANY:  0
IsoFit  FOUND
______________________________________
CHECKING : IT Service ArchiTechs
IS_COMPANY:  0
IT Service ArchiTechs  FOUND
______________________________________
CHECKING : Italian Hospital Group SpA
IS_COMPANY:  0
Italian Hospital Group SpA  FOUND
______________________________________
CHECKING : iwinks
IS_COMPANY:  0
iWinks  FOUND
______________________________________
CHECKING : Ixor
IS_COMPANY:  0
Ixor  FOUND
______________________________________
CHECKING : JAK Genetic and EMF Protection Svs
IS_COMPANY:  0
JAK Genetic and EMF Protection Svs  FOUND
______________________________________
CHECKING : Jan Medical
IS_COMPANY:  0
Jan Medical  FOUND

In [20]:
def writeObjectToFile(obj, filepath):
    with open(filepath, 'w') as outfile:
        json.dump(obj, outfile)
        outfile.close()

dct = {idx: Content_List[idx] for idx, name in enumerate(Content_List)}
writeObjectToFile(dct, 'content_list_chris1.json')

In [16]:
tab_data = tablib.Dataset()
data_str = open('content_list_chris.json').read()
data = json.loads(data_str)

In [17]:
for i in range(len(data)):
    if data[str(i)]['content'] == "FOUND":
        tab_data.append([ data[str(i)]['company'], data[str(i)]['content'], data[str(i)]['operating status'],
                    data[str(i)]['description'], data[str(i)]['funding status'], data[str(i)]['website'],
                    data[str(i)]['last funding type'], data[str(i)]['company type'], data[str(i)]['categories']])
    else:
        tab_data.append([ data[str(i)]['company'], data[str(i)]['content'],'N/A','N/A','N/A','N/A','N/A','N/A','N/A'])
    tab_data.headers = ['Company', 'Content', 'Operating Status', 'Description', 'Funding Status', 'Website', 
                        'Last Funding Type', 'Company Type', 'Categories' ]
    
csv_data = tab_data.export('csv')
f = open("company_chris.csv", "w")
f.write(csv_data)
f.close()